In [1]:
from langchain.vectorstores import Redis
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


import openai
import pandas as pd
import os
import redis

In [2]:
api_type = "azure"
api_base_url = "https://openaidemonubiral.openai.azure.com/"
api_version = "2023-03-15-preview"
azure_api_key = "ff5c606c134e4d1dae3426a412df834a"

openai.api_type = api_type
openai.api_base = api_base_url
openai.api_version = api_version
openai.api_key = azure_api_key

os.environ["OPENAI_API_BASE"] = api_base_url
os.environ["OPENAI_API_KEY"] = azure_api_key

#openai.Deployment.list()

In [3]:
embeddings_azure = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

llm_azure_chatgpt = AzureChatOpenAI(
    deployment_name="nubiral-lab-01", 
    temperature=0, 
    openai_api_version=api_version)

In [4]:
test_embedding = embeddings_azure.embed_query('test de emmbeding')
len(test_embedding)

1536

In [5]:
llm_azure_chatgpt([HumanMessage(content="Translate this sentence from English to French. I love programming.")])

AIMessage(content="J'aime programmer.", additional_kwargs={}, example=False)

In [6]:
myHostname = "vectordb.eastus2.redisenterprise.cache.azure.net"
myPassword = "7faRnxC4vBcEWoGNuB9hN6gnXxdYpdRtM5KhcGRq0d8="

r = redis.StrictRedis(host=myHostname, port=10000,
                      password=myPassword, ssl=True)

r.ping()

True

In [7]:
redis_url = "rediss://:7faRnxC4vBcEWoGNuB9hN6gnXxdYpdRtM5KhcGRq0d8=@vectordb.eastus2.redisenterprise.cache.azure.net:10000"
r = redis.from_url(url=redis_url)
r.ping()

True

In [8]:
loader = PyPDFLoader("../Data/Implementación de aprendizaje por refuerzo profundo para el control de sistemas dinámicos no lineales.pdf")

chunk_size = 1024
chunk_overlap = 200

text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    length_function = len,
)

pages = loader.load_and_split(text_splitter)

In [9]:
pages[0]

Document(page_content='1 \nResumen – El aprendizaje por refuerzo profundo, o Deep \nReinforcement Learning , es uno de los campos de la inteligencia \nartificial que tiene un potencial uso en los próximos años. Este \nparadigma p ermite que los algoritmos aprendan de su entorno \npara lograr objetivos, superando así las limitaciones que se \nencuentran en los algoritmos tradicionales de aprendizaje \nautomático  enfocas en  que las máquinas aprendan por sí solas.  Sin \nembargo, no existe un camino sistemático universal  a la hora de \nestablecer una metodología adecuada para abordar  los problemas \ny sus aplicaciones. Este articulo presenta  el uso de los algoritmos \nDeep Q -Learning  con un buffer de repetición de experiencias  \nimplementado en  Python utilizando las librerías de Tensorflow . \nComo primer  paso se establece el modelo  de un sistema dinámico \nno lineal ( péndulo invertido ) como el entorno de interacción para \nel agente de aprendiza je por refuerzo. El agente de

In [10]:
redis_url = "rediss://:7faRnxC4vBcEWoGNuB9hN6gnXxdYpdRtM5KhcGRq0d8=@vectordb.eastus2.redisenterprise.cache.azure.net:10000"
index_name = 'test_01'

rds = Redis.from_documents(pages, embeddings_azure, redis_url=redis_url,  index_name=index_name)

In [11]:
k=5
retriever_db = rds.as_retriever(search_type="similarity", search_kwargs={"k": k})

In [12]:
retriever_db.get_relevant_documents("quienes son los autores?")

[Document(page_content='los algoritmos de aprendizaje en muchos campos donde se \nrequiere lograr la autonomía del agente o sistema, fren te a los \ncontroladores clásicos dada la capacidad para generalizar \nexperiencia y extraer patrones de información  de las redes \nneuronales . \n \nÍndice de Términos - Deep Q Learning, péndulo invertido, control \nno lineal, redes neuronales, inteligencia artificial.  \n \nI. INTRODUCCI ÓN \n \n esde hace  varias  décadas la  respuesta  dinámica y el control \nde sistemas basados en n-péndulos invertidos ha tenido una \ngran relevancia  en el campo de la teoría de control  [4]. Los \ncontroladores tradicionales u otros reguladores se utilizan \nampliamente debido a su simplicidad y eficiencia. Sin embargo, estos \nrequieren de un modelo matemático preciso, además presentan \nmúltiples problemas cuando los modelos presentan una alta no \nlinealidad y son inestables en el tiempo [5]. Por lo que a raíz de estos \nproblemas surge la motivación de imp

In [13]:
retriever_db.get_relevant_documents("quienes son los autores?")

[Document(page_content='los algoritmos de aprendizaje en muchos campos donde se \nrequiere lograr la autonomía del agente o sistema, fren te a los \ncontroladores clásicos dada la capacidad para generalizar \nexperiencia y extraer patrones de información  de las redes \nneuronales . \n \nÍndice de Términos - Deep Q Learning, péndulo invertido, control \nno lineal, redes neuronales, inteligencia artificial.  \n \nI. INTRODUCCI ÓN \n \n esde hace  varias  décadas la  respuesta  dinámica y el control \nde sistemas basados en n-péndulos invertidos ha tenido una \ngran relevancia  en el campo de la teoría de control  [4]. Los \ncontroladores tradicionales u otros reguladores se utilizan \nampliamente debido a su simplicidad y eficiencia. Sin embargo, estos \nrequieren de un modelo matemático preciso, además presentan \nmúltiples problemas cuando los modelos presentan una alta no \nlinealidad y son inestables en el tiempo [5]. Por lo que a raíz de estos \nproblemas surge la motivación de imp

In [14]:
chain_type = "stuff"

qa = RetrievalQA.from_chain_type(llm=llm_azure_chatgpt, chain_type=chain_type, retriever=retriever_db, return_source_documents=True)

In [15]:
query = "quienes son los autores?"
result = qa({"query": query})

In [16]:
result['result']

'Los autores son C. H. Cristancho Toloza, J. A. Tumialán Borja, C. H. Rodríguez Garavito y D. J. Lancheros Cuestas.'

In [17]:
result['source_documents']

[Document(page_content='los algoritmos de aprendizaje en muchos campos donde se \nrequiere lograr la autonomía del agente o sistema, fren te a los \ncontroladores clásicos dada la capacidad para generalizar \nexperiencia y extraer patrones de información  de las redes \nneuronales . \n \nÍndice de Términos - Deep Q Learning, péndulo invertido, control \nno lineal, redes neuronales, inteligencia artificial.  \n \nI. INTRODUCCI ÓN \n \n esde hace  varias  décadas la  respuesta  dinámica y el control \nde sistemas basados en n-péndulos invertidos ha tenido una \ngran relevancia  en el campo de la teoría de control  [4]. Los \ncontroladores tradicionales u otros reguladores se utilizan \nampliamente debido a su simplicidad y eficiencia. Sin embargo, estos \nrequieren de un modelo matemático preciso, además presentan \nmúltiples problemas cuando los modelos presentan una alta no \nlinealidad y son inestables en el tiempo [5]. Por lo que a raíz de estos \nproblemas surge la motivación de imp

In [11]:
# Only redis

redis_url = "redis://redis_server:6379"
index_name = 'test_01'

rds_test = Redis(redis_url=redis_url,  index_name=index_name, embedding_function=embeddings_azure.embed_query)

In [12]:
k=5
retriever_db_test = rds_test.as_retriever(search_type="similarity", search_kwargs={"k": k})

In [14]:
retriever_db_test.get_relevant_documents("quienes son los autores?")

[Document(page_content='los algoritmos de aprendizaje en muchos campos donde se \nrequiere lograr la autonomía del agente o sistema, fren te a los \ncontroladores clásicos dada la capacidad para generalizar \nexperiencia y extraer patrones de información  de las redes \nneuronales . \n \nÍndice de Términos - Deep Q Learning, péndulo invertido, control \nno lineal, redes neuronales, inteligencia artificial.  \n \nI. INTRODUCCI ÓN \n \n esde hace  varias  décadas la  respuesta  dinámica y el control \nde sistemas basados en n-péndulos invertidos ha tenido una \ngran relevancia  en el campo de la teoría de control  [4]. Los \ncontroladores tradicionales u otros reguladores se utilizan \nampliamente debido a su simplicidad y eficiencia. Sin embargo, estos \nrequieren de un modelo matemático preciso, además presentan \nmúltiples problemas cuando los modelos presentan una alta no \nlinealidad y son inestables en el tiempo [5]. Por lo que a raíz de estos \nproblemas surge la motivación de imp

In [23]:
chain_type = "stuff"

qa = RetrievalQA.from_chain_type(llm=llm_azure_chatgpt, chain_type=chain_type, retriever=retriever_db_test, return_source_documents=True)

In [24]:
query = "quienes son los autores?"
result = qa({"query": query})

result['result']

'Los autores son C. H. Cristancho Toloza, J. A. Tumialán Borja, C. H. Rodríguez Garavito y D. J. Lancheros Cuestas.'

In [16]:
rds_test.client.close()